## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,East London,ZA,-33.0153,27.9116,56.55,59,51,3.09,broken clouds
1,1,Albany,US,42.6001,-73.9662,67.15,92,92,3.20,overcast clouds
2,2,Mataura,NZ,-46.1927,168.8643,53.47,45,13,2.15,few clouds
3,3,Port Elizabeth,ZA,-33.9180,25.5701,46.09,75,21,6.35,few clouds
4,4,Samarai,PG,-10.6167,150.6667,78.48,84,100,19.01,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Samarai,PG,-10.6167,150.6667,78.48,84,100,19.01,overcast clouds
6,6,Kapaa,US,22.0752,-159.3190,82.38,71,97,5.01,overcast clouds
7,7,Kavieng,PG,-2.5744,150.7967,83.77,76,32,14.23,light rain
9,9,Boa Vista,BR,2.8197,-60.6733,78.78,78,20,3.44,few clouds
20,20,Marzuq,YE,14.4000,46.4667,81.86,34,82,2.28,broken clouds
21,21,Ormara,PK,25.2088,64.6357,81.73,79,4,7.02,clear sky
27,27,Souillac,MU,-20.5167,57.5167,75.36,79,14,4.76,few clouds
29,29,Butaritari,KI,3.0707,172.7902,82.51,73,15,11.90,few clouds
30,30,Georgetown,MY,5.4112,100.3354,87.73,70,20,4.61,few clouds
32,32,Airai,TL,-8.9266,125.4092,77.99,27,38,7.40,scattered clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.replace("",float("NaN")).dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
682,682,Bandarban,BD,22.1965,92.2262,79.45,94,100,2.17,light rain
683,683,Pimenta Bueno,BR,-11.6725,-61.1936,80.96,51,20,3.20,few clouds
688,688,Gao,ML,16.6362,1.6370,86.05,42,54,9.57,broken clouds
697,697,Kalianget,ID,-7.0500,113.9333,85.39,63,36,14.20,scattered clouds
699,699,Tura,IN,25.5198,90.2201,78.57,83,100,4.70,moderate rain


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Samarai,PG,78.48,overcast clouds,-10.6167,150.6667,
6,Kapaa,US,82.38,overcast clouds,22.0752,-159.3190,
7,Kavieng,PG,83.77,light rain,-2.5744,150.7967,
9,Boa Vista,BR,78.78,few clouds,2.8197,-60.6733,
20,Marzuq,YE,81.86,broken clouds,14.4000,46.4667,
21,Ormara,PK,81.73,clear sky,25.2088,64.6357,
27,Souillac,MU,75.36,few clouds,-20.5167,57.5167,
29,Butaritari,KI,82.51,few clouds,3.0707,172.7902,
30,Georgetown,MY,87.73,few clouds,5.4112,100.3354,
32,Airai,TL,77.99,scattered clouds,-8.9266,125.4092,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
print("Data Analysis Finished.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Data Analysis Finished.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.replace("",float("NaN")).dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Samarai,PG,78.48,overcast clouds,-10.6167,150.6667,Nuli Sapi
6,Kapaa,US,82.38,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
7,Kavieng,PG,83.77,light rain,-2.5744,150.7967,Nusa Island Retreat
9,Boa Vista,BR,78.78,few clouds,2.8197,-60.6733,Aipana Plaza Hotel
21,Ormara,PK,81.73,clear sky,25.2088,64.6357,Crystal Beach and Camping Resort Ormara
27,Souillac,MU,75.36,few clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
29,Butaritari,KI,82.51,few clouds,3.0707,172.7902,Isles Sunset Lodge
30,Georgetown,MY,87.73,few clouds,5.4112,100.3354,Cititel Penang
32,Airai,TL,77.99,scattered clouds,-8.9266,125.4092,Juvinal Martin
35,Ambon,ID,80.74,moderate rain,-3.6954,128.1814,Swiss-Belhotel Ambon


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))